In [40]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

# Initialization cell
import os
import subprocess

import pandas as pd
import numpy as np

path_to_NPDtools = '/home/letovesnoi/work/tools/NPDtools-2.4.0-Linux/'

data_dir = 'data'
gnps_library = os.path.join(data_dir, 'GNPS-LIBRARY.mgf')
# gnps_library = os.path.join(data_dir, 'tst.mgf')

spectra_dir = os.path.join(data_dir, 'spectra')
mols_dir = os.path.join(data_dir, 'mols')
fthr = os.path.join(data_dir, 'data.fthr')

reg_run_dir = os.path.join(data_dir, 'REG_RUN_GNPS')
fdr0_tsv = os.path.join(reg_run_dir, 'regrun_fdr0_complete.tsv')
spectra_rr_dir = os.path.join(data_dir, 'spectra_REG_RUN')
reg_run_df = pd.read_csv(fdr0_tsv, sep="\t")
grouped_df = reg_run_df.groupby(['SpecFile'])['LocalSpecIdx', 'Structure'].agg(list)
fthr_rr = os.path.join(data_dir, 'data_rr.fthr')

discrete_masses = np.linspace(0, 5000, num=50000, dtype='float64')

In [1]:
# create directory for MGF files
if os.path.exists(spectra_dir):
    subprocess.call('rm -r {}'.format(spectra_dir), shell=True)
os.mkdir(spectra_dir)

# create directory for Molfiles
if os.path.exists(mols_dir):
    subprocess.call('rm -r {}'.format(mols_dir), shell=True)
os.mkdir(mols_dir)

In [2]:
def get_header(fin):
    header = []
    line = fin.readline()
    while '=' in line:
        ind = line.find('=')
        header.append((line[:ind], line[ind + 1:].strip()))
        line = fin.readline()
    return header, line

def get_spectrum(fin):
    header, line = get_header(fin)
    intensity = []
    while line != 'END IONS\n':
        intensity.append(line.strip().split())
        line = fin.readline()
    return header, np.array(intensity, dtype='float64')

def get_spectrum_id(header):
    return header[18][1]

def write_spectrum(header, intensity, spectrum_path):
    with open(spectrum_path, 'a') as fout:
        fout.write('BEGIN IONS\n')
        for key, value in header:
            fout.write(key + '=' + value + '\n')
        for mass, abundance in intensity:
            fout.write(str(mass) + ' ' + str(abundance) + '\n')
        fout.write('END IONS\n')

def split_mgf(gnps_library):
    spectra_pathes = []
    with open(gnps_library, 'r') as fin:
        num_spectra = 0
        while True:
            line = fin.readline()
            if not line:
                break
            num_spectra += 1
            header, intensity = get_spectrum(fin)
            spectra_pathes.append(os.path.join(spectra_dir, get_spectrum_id(header) + '.mgf'))
            write_spectrum(header, intensity, spectra_pathes[-1])
    # print('{} spectra'.format(num_spectra))
    return spectra_pathes

In [3]:
peptidic = 0
smiles_errors = 0
known = 0
empty = 0

def get_smiles(header):
    return header[11][1]

def get_mol(spectrum_path):
    with open(spectrum_path, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        molfile = os.path.join(mols_dir, get_spectrum_id(header) + '.mol')
        # print('SMILES=' + get_smiles(header))
        # print('SPECTRUMID=' + get_spectrum_id(header))
        subprocess.call('molconvert mol:V3+H -s \'{}\' -o {}'.format(get_smiles(header), molfile), shell=True)
    return molfile

def is_structural(mgf_file):
    global known
    with open(mgf_file, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        if get_smiles(header) == 'N/A' or get_smiles(header) == '':
            return False
        else:
            known += 1
            return True

def is_peptidic(molfile):
    global smiles_errors, peptidic
    tmp_peptidic = os.path.join(data_dir, 'peptidic.tmp')
    subprocess.call('{0}bin/print_structure {1} -C {0}share/npdtools/ --print_rule_fragmented_graph > {2}'.format(path_to_NPDtools, molfile, tmp_peptidic), shell=True)
    with open(tmp_peptidic, 'r') as fin:
        line = fin.readline()
        subprocess.call('rm {}'.format(tmp_peptidic), shell=True)
        if 'number of components' in line:
            num_components = line.strip().split(':')[1][1:]
        else:
            smiles_errors += 1
            print('{} contains an error'.format(molfile))
            return -1
    if int(num_components) > 3:
        peptidic += 1
        return 1
    else:
        return 0

def is_empty(mgf_file):
    global empty
    with open(mgf_file, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        if len(intensity) != 0:
            return False
        else:
            empty += 1
            print('{} has no information about intensity'.format(mgf_file))
            return True
        
def filter_spectra():
    for mgf_file in tqdm(split_mgf(gnps_library)):
        # remove unknown spectrum
        if not is_structural(mgf_file) or is_empty(mgf_file):
            subprocess.call('rm {}'.format(mgf_file), shell=True)
        else:
            molfile = get_mol(mgf_file)
            # remove non-peptidic data
            is_pep = is_peptidic(molfile)
            if is_pep == 0 or is_pep == -1:
                subprocess.call('rm {}'.format(mgf_file), shell=True)
                subprocess.call('rm {}'.format(molfile), shell=True)
                
filter_spectra()

print('{} have a known structure'.format(known))
print('{} have no information about intensity'.format(empty))
print('{} SMILES contains an error'.format(smiles_errors))
print('{} peptidic among structural, with non zero intensity, and with correct SMILES'.format(peptidic))

data/spectra/CCMSLIB00000078897.mgf has no information about intensity
data/mols/CCMSLIB00000478055.mol contains an error
data/mols/CCMSLIB00000478057.mol contains an error
data/mols/CCMSLIB00000478082.mol contains an error
data/mols/CCMSLIB00000478089.mol contains an error
data/mols/CCMSLIB00000478097.mol contains an error
data/mols/CCMSLIB00000478103.mol contains an error
data/spectra/CCMSLIB00000577483.mgf has no information about intensity
data/spectra/CCMSLIB00000577484.mgf has no information about intensity
data/spectra/CCMSLIB00000577485.mgf has no information about intensity
data/spectra/CCMSLIB00004722219.mgf has no information about intensity
data/mols/CCMSLIB00005435738.mol contains an error
data/mols/CCMSLIB00005436087.mol contains an error

2247 have a known structure
5 have no information about intensity
8 SMILES contains an error
443 peptidic among structural, with non zero intensity, and with correct SMILES


In [4]:
# linear, cyclic, branch-cyclic or complex
def get_fthr(spectra_pathes, fthr):   
    global discrete_masses
    spectra_df = pd.DataFrame()
    for mgf_file in tqdm(spectra_pathes):
        with open(mgf_file, 'r') as fin:
            fin.readline()
            header, intensity = get_spectrum(fin)
        id = os.path.splitext(os.path.basename(mgf_file))[0]
        bins = pd.cut(intensity[:, 0], bins=discrete_masses).astype(str)
        spectrum_df = pd.DataFrame({id: intensity[:, 1], 'binned': bins}).groupby(['binned']).sum().T
        spectra_df = pd.concat([spectra_df, spectrum_df], sort=True)
    spectra_df.reset_index().to_feather(fthr)
    return fthr

fthr = get_fthr([os.path.join(spectra_dir, spectrum_path) for spectrum_path in os.listdir(spectra_dir)], fthr)
# spectra_df = pd.read_feather(fthr)
# display(spectra_df)

/home/letovesnoi/tools/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,index,"(100.002, 100.102]","(100.102, 100.202]","(100.202, 100.302]","(100.302, 100.402]","(100.402, 100.502]","(100.502, 100.602]","(100.602, 100.702]","(100.702, 100.802]","(100.802, 100.902]",...,"(999.02, 999.12]","(999.12, 999.22]","(999.22, 999.32]","(999.32, 999.42]","(999.42, 999.52]","(999.52, 999.62]","(999.62, 999.72]","(999.72, 999.82]","(999.82, 999.92]","(999.92, 1000.02]"
0,CCMSLIB00000839203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CCMSLIB00000840535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CCMSLIB00005435708,36082.316406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CCMSLIB00000075335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,262.115692,NaN,NaN,NaN,NaN,NaN
4,CCMSLIB00003739971,224.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,CCMSLIB00004680107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,CCMSLIB00000425025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,CCMSLIB00004680121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,CCMSLIB00004680105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


85 linear
82 cyclic
71 branch-cyclic
205 complex


In [5]:
def split_mgf_by_ids(mgf_library, spectra_dir, local_ids):
    spectra_pathes = []
    with open(mgf_library, 'r') as fin:
        num_spectra = 0
        while True:
            line = fin.readline()
            if not line:
                break
            header, intensity = get_spectrum(fin)
            if num_spectra in local_ids:
                mgf_name = os.path.splitext(os.path.basename(mgf_library))[0]
                spectra_pathes.append(os.path.join(spectra_dir, mgf_name + '_' + str(num_spectra) + '.mgf'))
                write_spectrum(header, intensity, spectra_pathes[-1])
            num_spectra += 1
    return spectra_pathes

# create directory for MGF files
if os.path.exists(spectra_rr_dir):
    subprocess.call('rm -r {}'.format(spectra_rr_dir), shell=True)
os.mkdir(spectra_rr_dir)

spectra_rr_pathes = []
for index, row in grouped_df.iterrows():
    mgf_path = os.path.join(reg_run_dir, os.path.splitext(index)[0] + '_filtered.mgf')
    spectra_rr_pathes.extend(split_mgf_by_ids(mgf_path, spectra_rr_dir, row['LocalSpecIdx']))
    
fthr_rr = get_fthr(spectra_rr_pathes, fthr_rr)
# spectra_rr_df = pd.read_feather(fthr_rr)
# display(spectra_rr_df)

,index,"(100.002, 100.102]","(100.102, 100.202]","(100.202, 100.302]","(100.302, 100.402]","(100.402, 100.502]","(100.502, 100.602]","(100.602, 100.702]","(100.702, 100.802]","(100.802, 100.902]",...,"(999.02, 999.12]","(999.12, 999.22]","(999.22, 999.32]","(999.32, 999.42]","(999.42, 999.52]","(999.52, 999.62]","(999.62, 999.72]","(999.72, 999.82]","(999.82, 999.92]","(999.92, 1000.02]"
0,MSV000078556_filtered_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSV000078556_filtered_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSV000078556_filtered_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MSV000078556_filtered_3,9.485095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MSV000078556_filtered_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7500,MSV000080116_filtered_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7501,MSV000080116_filtered_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7502,MSV000080116_filtered_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7503,MSV000080116_filtered_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0 linear
0 cyclic
0 branch-cyclic
0 complex


In [24]:
# Skip other cells except initialization and run this
spectra_df = pd.read_feather(fthr)
spectra_rr_df = pd.read_feather(fthr_rr)
df = pd.concat([spectra_df, spectra_rr_df], sort=True)
del df['index']

/home/letovesnoi/tools/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [34]:
# Get labels
def get_spectrum_cyclicality(molfile):
    tmp_cyclicality = os.path.join(data_dir, 'cyclicality.tmp')
    subprocess.call('{0}bin/print_structure {1} -C {0}share/npdtools/ --print_structure > {2}'.format(path_to_NPDtools, molfile, tmp_cyclicality), shell=True)
    with open(tmp_cyclicality, 'r') as fin:
        cyclicality = fin.readline().strip()
    subprocess.call('rm {}'.format(tmp_cyclicality), shell=True)
    return cyclicality

cycl_dict = {'linear': 0, 'cyclic': 1, 'b-cyclic': 2, 'branch-cyclic': 2, 'complex': 3}

def get_labels(mols_dir):
    labels = []
    for molfile in tqdm(os.listdir(mols_dir)):
        cyclicality = get_spectrum_cyclicality(os.path.join(mols_dir, molfile))
        labels.append(cycl_dict[cyclicality])
    return labels    

def get_labels_rr(grouped_df):
    labels = []
    for index, row in grouped_df.iterrows():
        labels.extend([cycl_dict[cyclicality] for cyclicality in row['Structure']])
    return labels

labels = get_labels(mols_dir) + get_labels_rr(grouped_df)

print('{} linear'.format(labels.count(0)))
print('{} cyclic'.format(labels.count(1)))
print('{} branch-cyclic'.format(labels.count(2)))
print('{} complex'.format(labels.count(3)))


3186 linear
2763 cyclic
1763 branch-cyclic
236 complex


In [127]:
import keras

from sklearn.model_selection import train_test_split

X = df.to_numpy().astype('float32')
y = np.array(labels).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape)

(5325, 23325, 1)


In [132]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
# model.add(Conv2D(filters=4, kernel_size=4, padding='same', activation='relu', input_shape=(1, 23325, 1)))
# model.add(MaxPooling2D(pool_size=(1, 2)))
# model.add(Conv2D(filters=4, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(1,2)))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu', input_shape=(23325, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_39 (Conv1D)           (None, 23325, 32)         160       
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 11662, 32)         0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 11662, 64)         8256      
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 5831, 64)          0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 5831, 64)          0         
_________________________________________________________________
flatten_32 (Flatten)         (None, 373184)            0         
_________________________________________________________________
dense_39 (Dense)             (None, 64)                23883840  
__________

In [133]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [134]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=20, 
                 validation_split=0.2, callbacks=[checkpointer], verbose=2, shuffle=True)

Train on 4792 samples, validate on 533 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[373184,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_39/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4192094, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_37/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_39/random_uniform/RandomUniform', defined at:
  File "/home/letovesnoi/tools/anaconda3/envs/tensorflow_gpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/letovesnoi/tools/anaconda3/envs/tensorflow_gpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/letovesnoi/tools/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <mo

In [115]:
# load the weights that yielded the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [117]:
# evaluate and print test accuracy
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.3949675942326247
